In [68]:
import gzip
import json
import time
#import simplejson as json
import sys
import os
import glob
import datetime
from datetime import datetime




def cleanText(x):
    return not (x.startswith("#")|x.startswith("@")|x.startswith("http:")| (x == '\n')| (x == '\r'))

def getUnixTimeStamp(stamp):
    d = datetime.strptime(stamp,'%a %b %d %H:%M:%S +0000 %Y')
    unixtime = time.mktime(d.timetuple())
    return unixtime


def parseJson(jsonfile):
    tweets = []

    with gzip.open(jsonfile,'r') as fin:
        for line in fin:

            if len(line) > 1000: # check if the line is proper tweet (by observation, all valid tweets have length at least 1000.)
                d = json.loads(line.decode("UTF-8"))  # more effecient way would be check the line validity before loading

                if len(d['entities']['hashtags']): ## explicit booleaness to check is a list is empty
        
                    ## dow = day of week.  unpacking timestamp into individual vars 
                    ##(or maybe I should do this in the next phase?  note this is still in unicode)
                    dow, month, day, time,_,year = d['created_at'].split(" ") 
                    ## Assign the extracted values into a new json to be stored.
                                 ## UserName --> from_user
                    hash_list = ['#'+hash_string['text'] for hash_string in d['entities']['hashtags']]
                    mention_list = d['entities']['user_mentions']
                    #term_list = list(set(d['text'].split(" ")) - set(hash_list))
                    term_list = filter(cleanText, d['text'].split(" "))
                    
                    processed = {"from_user":d['user']['screen_name'],
                                 ## Split hashtag, we only want the text in hashtag, discard indices.
                                 "hashtag":hash_list, 
                                 ## Split terms in tweet text, remove \n and \r
                                 #"term": [w.replace('\n', '').replace('\r','') for w in d['text'].split(" ")], 
                                 "term": term_list, 
                                 ## append loc_ to each word in location
                                 #"location":['loc_' + s for s in d['user']['location'].split(" ")],
                                 "location":d['user']['location']
                                 ## mention ids
                                 "mention":d['entities']['user_mentions'],
                                 ## language
                                 "lang":d['user']['lang'],
                                 ## day of week
                                 "dow":dow,
                                 ## month in 3 letters
                                 "month": month,
                                 ## numeric day 1-31
                                 "day":day,
                                 ## hr:min:sec
                                 "time":time,
                                 ## numeric year
                                 "year":year,
                                 "created_at":getUnixTimeStamp(d['created_at'])
                                }
                    tweets.append(processed)

    return tweets




In [61]:


if __name__ == "__main__":

    #print(sys.argv)
    start_time = time.time()

    ## Linux
    #jsonfilename = '/home/danielshi/tw_pyspark/Sample/statuses.log.2014-07-01-00.gz'
    folder = 'F:/Demoonism/DataScientist/Spark/data/batch1' #sys.argv[1]    #'/mnt/f/Demoonism/Data Scientist/Spark/statuses.log.2014-07-01-00.gz'
    # /mnt/1e69d2b1-91a9-473c-a164-db90daf43a3d/Backup_tw_2013_2_8/2013-02/2013-02-01
    print("running folder ",folder, "..." )
    ## Windows
    #jsonfilename = '../statuses.log.2014-07-01-00.gz'
    path = os.path.join(folder,'*.gz')
    output = []
    files = glob.glob(path)

    counter = 0
    for jsonfilename in files:

        result = parseJson(jsonfilename)
        output = output + result  ##simple list concatenation
        counter += 1
        if counter % 2 == 0:
            print "processed ", counter
    out_filename = sys.argv[2]
    sec = time.time() - start_time
    m, s = divmod(sec, 60)
    h, m = divmod(m, 60)
    print('Pre-processing '+ folder +' takes - %d:%02d:%02d which is %d seconds in total' % (h,m,s,sec))
    with gzip.open(out_filename+'.json.gz', 'wb') as f:
        for eachjson in output:
            json.dump(eachjson, f) # works for json, cannot write(json)
            f.write('\n')

('running folder ', 'F:/Demoonism/DataScientist/Spark/data/batch1', '...')


TypeError: unsupported operand type(s) for |: 'bool' and 'unicode'

In [69]:


if __name__ == "__main__":

    #print(sys.argv)
    start_time = time.time()

    ## Linux
    #jsonfilename = '/home/danielshi/tw_pyspark/Sample/statuses.log.2014-07-01-00.gz'
    folder = 'F:/Demoonism/DataScientist/Spark/data/batch1' #sys.argv[1]    #'/mnt/f/Demoonism/Data Scientist/Spark/statuses.log.2014-07-01-00.gz'
    # /mnt/1e69d2b1-91a9-473c-a164-db90daf43a3d/Backup_tw_2013_2_8/2013-02/2013-02-01
    print("running folder ",folder, "..." )
    ## Windows
    #jsonfilename = '../statuses.log.2014-07-01-00.gz'
    path = os.path.join(folder,'*.gz')
    output = []
    files = glob.glob(path)

    counter = 0
    for jsonfilename in files:

        result = parseJson(jsonfilename)
        output = output + result  ##simple list concatenation
        counter += 1
        if counter % 2 == 0:
            print "processed ", counter
            
    print('done')


('running folder ', 'F:/Demoonism/DataScientist/Spark/data/batch1', '...')
done


In [70]:
print(output[4])

{'hashtag': [u'#SlowJama', u'#HarlemNights'], 'year': u'2014', 'month': u'Jun', 'mention': [], 'from_user': u'Gorgeous_Lady2', 'day': u'30', 'lang': u'en', 'term': [u'This', u'Thursday', u'18+', u'(Biggest', u'Lingerie/Pajama', u'Party)', u'Free', u'Til', u'12', u'+', u'Ladies', u'FREE', u'in', u'Lingerie', u'x11'], 'created_at': 1404151200.0, 'location': [u'loc_@Gorgeous_Lady2', u'loc_follows', u'loc_you'], 'time': u'14:00:00', 'dow': u'Mon'}


In [53]:
'http://t.co/RFPXOc1OEe'.startswith('http:')

True

In [9]:

#datetime_object = datetime.strptime(output[0]['created_at'],'%a %b %d %H:%M:%S %Z %Y')

d = datetime.strptime(output[4]['created_at'],'%a %b %d %H:%M:%S +0000 %Y')

d


datetime.datetime(2014, 6, 30, 14, 0)

In [10]:
unixtime = time.mktime(d.timetuple())

In [11]:
d.timetuple()

time.struct_time(tm_year=2014, tm_mon=6, tm_mday=30, tm_hour=14, tm_min=0, tm_sec=0, tm_wday=0, tm_yday=181, tm_isdst=-1)

In [12]:
import calendar
from datetime import datetime
d = datetime.utcnow()
timestamp=calendar.timegm(output[0]['created_at'].encode('ascii','ignore').utctimetuple())

AttributeError: 'str' object has no attribute 'utctimetuple'

In [14]:
unixtime

1404151200.0

In [16]:
import re,string

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)


tests = [
    "@peter I really love that shirt at #Macy. http://bet.ly//WjdiW4",
    "@shawn Titanic tragedy could have been prevented Economic Times: Telegraph.co.ukTitanic tragedy could have been preve... http://bet.ly/tuN2wx",
    "I am at Starbucks http://4sh.com/samqUI (7419 3rd ave, at 75th, Brooklyn)",
]
for t in tests:
    print(strip_all_entities(strip_links(t)))


#'I really love that shirt at'
#'Titanic tragedy could have been prevented Economic Times Telegraph co ukTitanic tragedy could have been preve'
#'I am at Starbucks 7419 3rd ave at 75th Brooklyn'

I really love that shirt at
Titanic tragedy could have been prevented Economic Times Telegraph co ukTitanic tragedy could have been preve
I am at Starbucks 7419 3rd ave at 75th Brooklyn


In [25]:
list_1 = ['a', 'big', 'list']
list_2 = ['another', 'big', 'list']

intersection = list(set(list_1) - set(list_2))

In [57]:
'x'.equals('x')

AttributeError: 'str' object has no attribute 'equals'